# Environmental Conditions in Vienna

[![Binder](http://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/ogd-at-lab/main?urlpath=lab/tree/notebooks/environment.ipynb)




In [ ]:
import hvplot.pandas
import pandas as pd
import geopandas as gpd
from utils.dataaccess import get_gdf_from_wfs, get_airquality_df, get_heatvulnerabilityindex_gdf

## Urban Heat

Dataset: [Urban Heat Vulnerability Index (UHVI)](https://www.data.gv.at/katalog/dataset/urban-heat-vulnerability-index-uhvi-wien)

In [ ]:
uhvi = get_heatvulnerabilityindex_gdf()

In [ ]:
uhvi.hvplot(geo=True, tiles='OSM', c='AVG_UHVI_A', title='Average Urban Heat Vulnerability Index - All age groups', cmap='Reds', alpha=0.7).opts(active_tools=['wheel_zoom'])

## Air Quality

Dataset: [Luftmessnetz: aktuelle Messdaten Wien](https://www.data.gv.at/katalog/dataset/luftmessnetz-aktuelle-messdaten-wien/)

In [ ]:
stations = get_gdf_from_wfs('LUFTGUETENETZOGD')
stations.set_index('NAME_KURZ', inplace=True)
stations.head()

In [ ]:
air_quality = get_airquality_df()
air_quality.head()

In [ ]:
col = 'airtemp °C'
stations = stations.join(air_quality).dropna(subset=[col])

In [ ]:
stations.hvplot(geo=True, tiles='StamenToner', c=col, size=500, title=col.title())

In [ ]:
from holoviews.selection import link_selections

In [ ]:
my_map = stations.hvplot(geo=True, tiles='CartoLight', c=col, cmap='Viridis', size=200, title=col.title(), width=400, height=250)
my_scatter = pd.DataFrame(stations).hvplot.scatter(x='airtemp °C', y='windspeed kmh', width=250, height=250)
my_hist = pd.DataFrame(stations).hvplot.hist('airtemp °C', width=250, height=250)
link_selections(my_map + my_hist + my_scatter , index_cols=['id'])

In [ ]:
import panel as pn

In [ ]:
def my_plot(my_col='airtemp °C'):
    try: 
        my_map = stations.hvplot(geo=True, tiles='CartoLight', c=my_col, cmap='Viridis', size=200, title=my_col, width=400, height=250)
        my_hist = pd.DataFrame(stations).hvplot.hist(my_col, title=my_col, width=250, height=250)
        return link_selections(my_map + my_hist, index_cols=['id']) 
    except:
        pass
kw = dict(my_col=air_quality.columns)
pn.interact(my_plot, **kw)